### Chapter 5: Modules and Interfaces

#### Module Basics

Modules are the fundamental building blocks of SystemVerilog designs. They encapsulate functionality and provide a way to create hierarchical designs through instantiation.

##### Basic Module Structure

```systemverilog
module module_name #(
    // Parameters (optional)
    parameter int WIDTH = 8
) (
    // Port declarations
    input  logic clk,
    input  logic reset_n,
    input  logic [WIDTH-1:0] data_in,
    output logic [WIDTH-1:0] data_out
);

    // Module body - internal logic
    always_ff @(posedge clk or negedge reset_n) begin
        if (!reset_n)
            data_out <= '0;
        else
            data_out <= data_in;
    end

endmodule
```

##### Module Instantiation

```systemverilog
// Named port connections (recommended)
module_name #(.WIDTH(16)) inst_name (
    .clk(system_clk),
    .reset_n(sys_reset),
    .data_in(input_data),
    .data_out(output_data)
);

// Positional port connections (not recommended for complex modules)
module_name #(16) inst_name (system_clk, sys_reset, input_data, output_data);
```

##### Key Module Concepts

**Scope and Hierarchy**: Each module creates its own scope. Internal signals and variables are not accessible from outside the module unless explicitly connected through ports.

**Instance vs Module**: A module is the template/definition, while an instance is a specific instantiation of that module in your design.

#### Port Declarations and Directions

SystemVerilog provides several ways to declare module ports, offering more flexibility than traditional Verilog.

##### Port Directions

```systemverilog
module port_example (
    input  logic        clk,           // Input port
    output logic        valid,         // Output port
    inout  wire         bidir_signal,  // Bidirectional port
    ref    int          shared_var     // Reference port (SystemVerilog)
);
```

##### ANSI-Style Port Declarations (Recommended)

```systemverilog
module counter #(
    parameter int WIDTH = 8
) (
    input  logic             clk,
    input  logic             reset_n,
    input  logic             enable,
    input  logic             load,
    input  logic [WIDTH-1:0] load_value,
    output logic [WIDTH-1:0] count,
    output logic             overflow
);

    logic [WIDTH-1:0] count_reg;
    
    always_ff @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            count_reg <= '0;
            overflow <= 1'b0;
        end else if (load) begin
            count_reg <= load_value;
            overflow <= 1'b0;
        end else if (enable) begin
            {overflow, count_reg} <= count_reg + 1'b1;
        end
    end
    
    assign count = count_reg;

endmodule
```

##### Non-ANSI Style (Legacy)

```systemverilog
module counter (clk, reset_n, enable, count);
    parameter WIDTH = 8;
    
    input             clk;
    input             reset_n;
    input             enable;
    output [WIDTH-1:0] count;
    
    // Port declarations separate from module header
endmodule
```

##### Advanced Port Features

**Interface Ports**:
```systemverilog
module processor (
    input logic clk,
    input logic reset_n,
    memory_if.master mem_bus,  // Interface port
    axi4_if.slave    axi_port
);
```

**Unpacked Array Ports**:
```systemverilog
module multi_port (
    input  logic [7:0] data_in [0:3],   // Array of inputs
    output logic [7:0] data_out [0:3]   // Array of outputs
);
```

#### Parameters and Localparams

Parameters provide a way to create configurable, reusable modules. They allow customization at instantiation time.

##### Parameter Types

```systemverilog
module parameterized_module #(
    // Type parameters
    parameter type DATA_TYPE = logic [31:0],
    parameter type ADDR_TYPE = logic [15:0],
    
    // Value parameters
    parameter int DATA_WIDTH = 32,
    parameter int ADDR_WIDTH = 16,
    parameter int DEPTH = 1024,
    
    // String parameters
    parameter string MODE = "NORMAL",
    
    // Real parameters
    parameter real FREQUENCY = 100.0
) (
    input  logic      clk,
    input  DATA_TYPE  data_in,
    input  ADDR_TYPE  address,
    output DATA_TYPE  data_out
);
```

##### Localparam Usage

Localparams are parameters that cannot be overridden during instantiation. They're typically used for derived values.

```systemverilog
module memory #(
    parameter int DATA_WIDTH = 32,
    parameter int ADDR_WIDTH = 10
) (
    input  logic                    clk,
    input  logic                    we,
    input  logic [ADDR_WIDTH-1:0]   addr,
    input  logic [DATA_WIDTH-1:0]   wdata,
    output logic [DATA_WIDTH-1:0]   rdata
);

    // Localparams derived from parameters
    localparam int DEPTH = 2**ADDR_WIDTH;
    localparam int BYTES_PER_WORD = DATA_WIDTH / 8;
    
    logic [DATA_WIDTH-1:0] mem_array [0:DEPTH-1];
    
    always_ff @(posedge clk) begin
        if (we)
            mem_array[addr] <= wdata;
        rdata <= mem_array[addr];
    end

endmodule
```

##### Parameter Override Examples

```systemverilog
// Override during instantiation
memory #(
    .DATA_WIDTH(64),
    .ADDR_WIDTH(12)
) ram_inst (
    .clk(clk),
    .we(write_enable),
    .addr(address),
    .wdata(write_data),
    .rdata(read_data)
);

// Using defparam (not recommended)
defparam ram_inst.DATA_WIDTH = 64;
defparam ram_inst.ADDR_WIDTH = 12;
```

#### Generate Blocks

Generate blocks allow you to create repetitive hardware structures and conditional compilation based on parameters.

##### Generate For Loops

```systemverilog
module parallel_adder #(
    parameter int WIDTH = 32,
    parameter int STAGES = 4
) (
    input  logic [WIDTH-1:0] a,
    input  logic [WIDTH-1:0] b,
    input  logic             cin,
    output logic [WIDTH-1:0] sum,
    output logic             cout
);

    localparam int BITS_PER_STAGE = WIDTH / STAGES;
    
    logic [STAGES:0] carry;
    assign carry[0] = cin;
    assign cout = carry[STAGES];
    
    // Generate multiple adder stages
    generate
        for (genvar i = 0; i < STAGES; i++) begin : adder_stage
            logic [BITS_PER_STAGE-1:0] stage_sum;
            logic                      stage_cout;
            
            full_adder #(.WIDTH(BITS_PER_STAGE)) fa_inst (
                .a(a[i*BITS_PER_STAGE +: BITS_PER_STAGE]),
                .b(b[i*BITS_PER_STAGE +: BITS_PER_STAGE]),
                .cin(carry[i]),
                .sum(stage_sum),
                .cout(stage_cout)
            );
            
            assign sum[i*BITS_PER_STAGE +: BITS_PER_STAGE] = stage_sum;
            assign carry[i+1] = stage_cout;
        end
    endgenerate

endmodule
```

##### Generate If-Else

```systemverilog
module configurable_memory #(
    parameter int    DATA_WIDTH = 32,
    parameter int    ADDR_WIDTH = 10,
    parameter string MEMORY_TYPE = "BLOCK"  // "BLOCK" or "DISTRIBUTED"
) (
    input  logic                    clk,
    input  logic                    we,
    input  logic [ADDR_WIDTH-1:0]   addr,
    input  logic [DATA_WIDTH-1:0]   wdata,
    output logic [DATA_WIDTH-1:0]   rdata
);

    localparam int DEPTH = 2**ADDR_WIDTH;
    
    generate
        if (MEMORY_TYPE == "BLOCK") begin : block_memory
            // Use block RAM
            logic [DATA_WIDTH-1:0] mem [0:DEPTH-1];
            
            always_ff @(posedge clk) begin
                if (we)
                    mem[addr] <= wdata;
                rdata <= mem[addr];
            end
            
        end else if (MEMORY_TYPE == "DISTRIBUTED") begin : dist_memory
            // Use distributed RAM
            logic [DATA_WIDTH-1:0] mem [0:DEPTH-1];
            
            always_ff @(posedge clk) begin
                if (we)
                    mem[addr] <= wdata;
            end
            
            assign rdata = mem[addr];  // Combinational read
            
        end else begin : error_memory
            // Generate compile-time error for invalid parameter
            initial begin
                $error("Invalid MEMORY_TYPE parameter: %s", MEMORY_TYPE);
            end
        end
    endgenerate

endmodule
```

##### Generate Case

```systemverilog
module priority_encoder #(
    parameter int WIDTH = 8
) (
    input  logic [WIDTH-1:0] data_in,
    output logic [$clog2(WIDTH)-1:0] encoded_out,
    output logic valid
);

    generate
        case (WIDTH)
            4: begin : enc_4bit
                always_comb begin
                    casez (data_in)
                        4'b???1: {valid, encoded_out} = {1'b1, 2'd0};
                        4'b??10: {valid, encoded_out} = {1'b1, 2'd1};
                        4'b?100: {valid, encoded_out} = {1'b1, 2'd2};
                        4'b1000: {valid, encoded_out} = {1'b1, 2'd3};
                        default: {valid, encoded_out} = {1'b0, 2'd0};
                    endcase
                end
            end
            
            8: begin : enc_8bit
                // Implementation for 8-bit encoder
                always_comb begin
                    casez (data_in)
                        8'b???????1: {valid, encoded_out} = {1'b1, 3'd0};
                        8'b??????10: {valid, encoded_out} = {1'b1, 3'd1};
                        8'b?????100: {valid, encoded_out} = {1'b1, 3'd2};
                        8'b????1000: {valid, encoded_out} = {1'b1, 3'd3};
                        8'b???10000: {valid, encoded_out} = {1'b1, 3'd4};
                        8'b??100000: {valid, encoded_out} = {1'b1, 3'd5};
                        8'b?1000000: {valid, encoded_out} = {1'b1, 3'd6};
                        8'b10000000: {valid, encoded_out} = {1'b1, 3'd7};
                        default:     {valid, encoded_out} = {1'b0, 3'd0};
                    endcase
                end
            end
            
            default: begin : enc_generic
                // Generic implementation for other widths
                always_comb begin
                    encoded_out = '0;
                    valid = 1'b0;
                    for (int i = 0; i < WIDTH; i++) begin
                        if (data_in[i]) begin
                            encoded_out = i[$clog2(WIDTH)-1:0];
                            valid = 1'b1;
                            break;
                        end
                    end
                end
            end
        endcase
    endgenerate

endmodule
```

#### Introduction to Interfaces

Interfaces provide a powerful way to group related signals and simplify connections between modules. They help reduce port lists and improve code maintainability.

##### Basic Interface Declaration

```systemverilog
interface memory_if #(
    parameter int DATA_WIDTH = 32,
    parameter int ADDR_WIDTH = 16
) (
    input logic clk,
    input logic reset_n
);

    // Interface signals
    logic                    valid;
    logic                    ready;
    logic                    we;
    logic [ADDR_WIDTH-1:0]   addr;
    logic [DATA_WIDTH-1:0]   wdata;
    logic [DATA_WIDTH-1:0]   rdata;
    logic                    error;
    
    // Tasks and functions can be defined in interfaces
    task write_transaction(
        input logic [ADDR_WIDTH-1:0] address,
        input logic [DATA_WIDTH-1:0] data
    );
        @(posedge clk);
        valid <= 1'b1;
        we <= 1'b1;
        addr <= address;
        wdata <= data;
        @(posedge clk);
        while (!ready) @(posedge clk);
        valid <= 1'b0;
        we <= 1'b0;
    endtask
    
    task read_transaction(
        input  logic [ADDR_WIDTH-1:0] address,
        output logic [DATA_WIDTH-1:0] data
    );
        @(posedge clk);
        valid <= 1'b1;
        we <= 1'b0;
        addr <= address;
        @(posedge clk);
        while (!ready) @(posedge clk);
        data = rdata;
        valid <= 1'b0;
    endtask

endinterface
```

##### Using Interfaces in Modules

```systemverilog
// Memory controller module
module memory_controller (
    memory_if.slave  cpu_if,    // CPU interface (slave perspective)
    memory_if.master mem_if     // Memory interface (master perspective)
);

    // Interface connection logic
    always_comb begin
        // Forward CPU requests to memory
        mem_if.valid = cpu_if.valid;
        mem_if.we    = cpu_if.we;
        mem_if.addr  = cpu_if.addr;
        mem_if.wdata = cpu_if.wdata;
        
        // Forward memory responses to CPU
        cpu_if.ready = mem_if.ready;
        cpu_if.rdata = mem_if.rdata;
        cpu_if.error = mem_if.error;
    end

endmodule

// Memory module
module memory (
    memory_if.slave mem_if
);

    localparam int DEPTH = 2**mem_if.ADDR_WIDTH;
    logic [mem_if.DATA_WIDTH-1:0] mem_array [0:DEPTH-1];
    
    always_ff @(posedge mem_if.clk or negedge mem_if.reset_n) begin
        if (!mem_if.reset_n) begin
            mem_if.ready <= 1'b0;
            mem_if.rdata <= '0;
            mem_if.error <= 1'b0;
        end else begin
            mem_if.ready <= mem_if.valid;
            mem_if.error <= 1'b0;
            
            if (mem_if.valid) begin
                if (mem_if.we) begin
                    mem_array[mem_if.addr] <= mem_if.wdata;
                end else begin
                    mem_if.rdata <= mem_array[mem_if.addr];
                end
            end
        end
    end

endmodule
```

##### Interface Instantiation and Connection

```systemverilog
module top_level;
    logic clk, reset_n;
    
    // Interface instances
    memory_if #(.DATA_WIDTH(32), .ADDR_WIDTH(16)) cpu_mem_if(clk, reset_n);
    memory_if #(.DATA_WIDTH(32), .ADDR_WIDTH(16)) ctrl_mem_if(clk, reset_n);
    
    // Module instances
    cpu cpu_inst (
        .clk(clk),
        .reset_n(reset_n),
        .mem_if(cpu_mem_if.master)  // CPU is master
    );
    
    memory_controller ctrl_inst (
        .cpu_if(cpu_mem_if.slave),   // Controller is slave to CPU
        .mem_if(ctrl_mem_if.master)  // Controller is master to memory
    );
    
    memory mem_inst (
        .mem_if(ctrl_mem_if.slave)   // Memory is slave
    );

endmodule
```

#### Modports and Clocking Blocks

Modports define different views of an interface for different modules, while clocking blocks provide synchronous timing control.

##### Modports

Modports specify which signals are inputs, outputs, or inouts from a particular module's perspective.

```systemverilog
interface axi4_lite_if #(
    parameter int DATA_WIDTH = 32,
    parameter int ADDR_WIDTH = 32
) (
    input logic aclk,
    input logic aresetn
);

    // Write Address Channel
    logic [ADDR_WIDTH-1:0]  awaddr;
    logic [2:0]             awprot;
    logic                   awvalid;
    logic                   awready;
    
    // Write Data Channel
    logic [DATA_WIDTH-1:0]  wdata;
    logic [(DATA_WIDTH/8)-1:0] wstrb;
    logic                   wvalid;
    logic                   wready;
    
    // Write Response Channel
    logic [1:0]             bresp;
    logic                   bvalid;
    logic                   bready;
    
    // Read Address Channel
    logic [ADDR_WIDTH-1:0]  araddr;
    logic [2:0]             arprot;
    logic                   arvalid;
    logic                   arready;
    
    // Read Data Channel
    logic [DATA_WIDTH-1:0]  rdata;
    logic [1:0]             rresp;
    logic                   rvalid;
    logic                   rready;
    
    // Master modport (drives address/data, receives responses)
    modport master (
        input  aclk, aresetn,
        output awaddr, awprot, awvalid,
        input  awready,
        output wdata, wstrb, wvalid,
        input  wready,
        input  bresp, bvalid,
        output bready,
        output araddr, arprot, arvalid,
        input  arready,
        input  rdata, rresp, rvalid,
        output rready
    );
    
    // Slave modport (receives address/data, drives responses)
    modport slave (
        input  aclk, aresetn,
        input  awaddr, awprot, awvalid,
        output awready,
        input  wdata, wstrb, wvalid,
        output wready,
        output bresp, bvalid,
        input  bready,
        input  araddr, arprot, arvalid,
        output arready,
        output rdata, rresp, rvalid,
        input  rready
    );
    
    // Monitor modport (all inputs for verification)
    modport monitor (
        input aclk, aresetn,
        input awaddr, awprot, awvalid, awready,
        input wdata, wstrb, wvalid, wready,
        input bresp, bvalid, bready,
        input araddr, arprot, arvalid, arready,
        input rdata, rresp, rvalid, rready
    );

endinterface
```

##### Clocking Blocks

Clocking blocks define synchronous timing relationships and provide a clean way to handle clocked signals in testbenches.

```systemverilog
interface processor_if (
    input logic clk,
    input logic reset_n
);

    logic [31:0] instruction;
    logic [31:0] pc;
    logic        valid;
    logic        ready;
    logic        stall;
    logic        flush;
    
    // Clocking block for testbench use
    clocking cb @(posedge clk);
        default input #1step output #2ns;  // Input skew and output delay
        
        input  pc, valid, ready;
        output instruction, stall, flush;
    endclocking
    
    // Separate clocking block for different timing requirements
    clocking slow_cb @(posedge clk);
        default input #5ns output #10ns;
        
        input  pc, valid;
        output instruction;
    endclocking
    
    // Modports with clocking blocks
    modport tb (
        clocking cb,
        input clk, reset_n
    );
    
    modport dut (
        input  clk, reset_n,
        output pc, valid, ready,
        input  instruction, stall, flush
    );

endinterface
```

##### Advanced Clocking Block Example

```systemverilog
interface memory_test_if (
    input logic clk,
    input logic reset_n
);

    logic [15:0] addr;
    logic [31:0] wdata;
    logic [31:0] rdata;
    logic        we;
    logic        re;
    logic        valid;
    logic        ready;
    
    // Clocking block with different timing for different signals
    clocking driver_cb @(posedge clk);
        default input #2ns output #1ns;
        
        output addr, wdata, we, re, valid;
        input  rdata, ready;
    endclocking
    
    // Monitor clocking block samples everything
    clocking monitor_cb @(posedge clk);
        default input #1step;
        
        input addr, wdata, rdata, we, re, valid, ready;
    endclocking
    
    // Synchronous reset clocking block
    clocking reset_cb @(posedge clk);
        input reset_n;
    endclocking
    
    modport driver (
        clocking driver_cb,
        input clk, reset_n
    );
    
    modport monitor (
        clocking monitor_cb,
        input clk, reset_n
    );
    
    modport dut (
        input  clk, reset_n,
        input  addr, wdata, we, re, valid,
        output rdata, ready
    );

endinterface
```

##### Using Clocking Blocks in Testbenches

```systemverilog
module memory_testbench;
    logic clk = 0;
    logic reset_n;
    
    always #5ns clk = ~clk;  // 100MHz clock
    
    memory_test_if mem_if(clk, reset_n);
    
    // DUT instantiation
    memory dut (
        .mem_if(mem_if.dut)
    );
    
    // Test program using clocking blocks
    initial begin
        reset_n = 0;
        ##2 reset_n = 1;  // Wait 2 clock cycles
        
        // Write operation using clocking block
        mem_if.driver_cb.addr  <= 16'h1000;
        mem_if.driver_cb.wdata <= 32'hDEADBEEF;
        mem_if.driver_cb.we    <= 1'b1;
        mem_if.driver_cb.valid <= 1'b1;
        
        ##1;  // Wait 1 clock cycle
        
        wait (mem_if.driver_cb.ready);  // Wait for ready
        
        mem_if.driver_cb.we    <= 1'b0;
        mem_if.driver_cb.valid <= 1'b0;
        
        ##2;  // Wait before read
        
        // Read operation
        mem_if.driver_cb.addr  <= 16'h1000;
        mem_if.driver_cb.re    <= 1'b1;
        mem_if.driver_cb.valid <= 1'b1;
        
        ##1;
        
        wait (mem_if.driver_cb.ready);
        
        $display("Read data: %h", mem_if.driver_cb.rdata);
        
        mem_if.driver_cb.re    <= 1'b0;
        mem_if.driver_cb.valid <= 1'b0;
        
        ##5;
        $finish;
    end

endmodule
```

#### Summary

This chapter covered the essential concepts of SystemVerilog modules and interfaces:

**Modules** form the basic building blocks with proper port declarations and hierarchical instantiation capabilities.

**Parameters and localparams** enable configurable and reusable designs with type safety and parameter validation.

**Generate blocks** provide powerful compile-time code generation for creating repetitive structures and conditional compilation.

**Interfaces** simplify complex designs by grouping related signals and providing reusable communication protocols.

**Modports** define different perspectives of interfaces for various modules, ensuring proper signal direction and access control.

**Clocking blocks** provide precise timing control for synchronous designs, particularly useful in verification environments.

These features work together to create scalable, maintainable, and reusable SystemVerilog designs that can handle complex digital systems efficiently.